In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     66   10    9  111.85  653  111.03  0.81  0.01  104.14  499  103.12   
10000    44   20   19  111.37  738  110.35  1.02  0.01  103.91  492  102.69   
15000    47   30   29  111.09  777  109.90  1.19  0.01  103.72  487  102.21   
20000    53   40   39  110.92  791  109.58  1.32  0.03  103.61  482  101.94   
25000    76   50   49  110.79  797  109.34  1.42  0.03  103.45  478  101.68   
30000    59   60   59  110.69  798  109.15  1.50  0.04  103.41  476  101.61   
35000    56   70   69  110.60  796  109.00  1.57  0.04  103.38  476  101.50   
40000    53   80   79  110.53  792  108.88  1.62  0.04  103.31  474  101.42   
45000    58   90   89  110.47  788  108.77  1.66  0.04  103.25  472  101.33   
50000    57  100   99  110.42  783  108.68  1.70  0.03  103.18  470  101.27   
55000    59  110  109  110.37  777  108.60  1.74  0.03  103.12  469  101.17   
60000    53  120  119  110.33  772  108.54  1.76  0.03  103.11  468  101.17   
65000    55  130  129  110.30  767  108.48  1.79  0.03  103.08  468  101.09   
70000    51  140  139  110.27  763  108.42  1.81  0.03  103.06  468  101.08   
75000    54  150  149  110.24  758  108.38  1.84  0.03  103.09  469  101.08   
80000    51  160  159  110.21  754  108.33  1.85  0.03  103.07  468  101.07   
85000    54  170  169  110.19  749  108.29  1.87  0.02  103.08  469  101.05   
90000    50  180  179  110.17  746  108.26  1.89  0.02  103.05  468  101.00   
95000    54  190  189  110.15  742  108.22  1.91  0.02  103.22  473  101.16   
100000   55  200  199  110.13  738  108.19  1.92  0.02  103.11  470  101.06   
105000   61  210  209  110.12  734  108.16  1.93  0.02  103.09  469  101.03   
110000   55  220  219  110.10  731  108.14  1.95  0.02  103.08  469  101.01   
115000   58  230  229  110.09  727  108.11  1.96  0.02  103.07  469  100.96   
120000   58  240  239  110.07  724  108.09  1.97  0.02  103.03  468  100.95   
125000   57  250  249  110.06  721  108.06  1.98  0.02  103.02  467  100.92   
130000   56  260  259  110.05  718  108.04  1.99  0.02  102.99  466  100.88   
135000   58  270  269  110.03  716  108.02  2.00  0.02  103.01  467  100.89   
140000   55  280  279  110.02  713  108.00  2.01  0.02  102.98  466  100.86   
145000   55  290  289  110.01  710  107.98  2.02  0.02  103.03  467  100.90   
150000   60  300  299  110.00  708  107.96  2.03  0.02  103.03  467  100.89   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   52  701  200  109.80  656  107.61  2.19  0.01  103.02  467  100.81   
355000   55  711  210  109.80  656  107.60  2.19  0.01  103.00  467  100.77   
360000   52  721  220  109.80  655  107.60  2.19  0.01  103.00  466  100.79   
365000   56  731  230  109.79  654  107.60  2.20  0.01  102.97  466  100.75   
370000   51  741  240  109.79  654  107.59  2.20  0.01  102.98  466  100.76   
375000   66  751  250  109.79  653  107.59  2.20  0.01  102.97  466  100.74   
380000   54  761  260  109.79  653  107.58  2.20  0.01  102.97  465  100.74   
385000   56  771  270  109.78  652  107.58  2.20  0.01  102.95  465  100.71   
390000   62  781  280  109.78  652  107.58  2.21  0.01  102.95  465  100.73   
395000   70  791  290  109.78  651  107.57  2.21  0.01  102.96  465  100.72   
400000   54  801  300  109.78  651  107.57  2.21  0.01  102.96  465  100.75   
405000   56  811  310  109.77  650  107.56  2.21  0.01  102.96  465  100.74   
410000   52  821  320  109.77  650  107.56  2.21  0.01  102.97  465  100.75   
415000   56  831  330  109.77  649  107.56  2.22  0.01  102.97  466  100.72   
420000   52  841  340  109.77  649  107.55  2.22  0.01  102.99  466  100.77   
425000   56  851  350  109.76  648  107.55  2.22  0.01  102.98  466  100.73   
430000   52  861  360  109.76  648  107.55  2.22  0.01  103.00  467  100.79   
435000   71  871  370  109.76  647  107.54  2.22  0.01  102.97  466  100.

0 R: 1.000 P: 0.394 quality price 'm bought - ... time nice made 've
   1 R: 0.295 P: 0.136 pocket room ipad small power mouse charger carry cord netbook
     11 R: 0.084 P: 0.084 pockets carry strap shoulder compartment straps pack handle back compartments
     14 R: 0.036 P: 0.036 ! love room carry perfect recommend plenty highly pockets easy
     12 R: 0.039 P: 0.039 ! love school carry bought books college recommend work quality
   4 R: 0.108 P: 0.072 sleeve protection air inside inch pro protect neoprene snug chromebook
     41 R: 0.036 P: 0.036 & ; sleeve inside perfectly pro size hp big dell
   5 R: 0.090 P: 0.044 mac cover pro love perfectly recommend ! protect hard color
     52 R: 0.046 P: 0.046 ! love color perfect price perfectly buy ... recommend cute
   2 R: 0.113 P: 0.068 cover bottom keyboard top screen hard plastic speck easily air
     21 R: 0.045 P: 0.045 color picture cover love ordered blue ! received pink keyboard
{0: [1, 4, 2, 3], 1: [11, 13], 4: [42], 2: [22], 3

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     66   10    9  111.85  653  111.03  0.81  0.01  104.14  499  103.12   
10000    44   20   19  111.37  738  110.35  1.02  0.01  103.91  492  102.69   
15000    47   30   29  111.09  777  109.90  1.19  0.01  103.72  487  102.21   
20000    53   40   39  110.92  791  109.58  1.32  0.03  103.61  482  101.94   
25000    76   50   49  110.79  797  109.34  1.42  0.03  103.45  478  101.68   
30000    59   60   59  110.69  798  109.15  1.50  0.04  103.41  476  101.61   
35000    56   70   69  110.60  796  109.00  1.57  0.04  103.38  476  101.50   
40000    53   80   79  110.53  792  108.88  1.62  0.04  103.31  474  101.42   
45000    58   90   89  110.47  788  108.77  1.66  0.04  103.25  472  101.33   
50000    57  100   99  110.42  783  108.68  1.70  0.03  103.18  470  101.27   
55000    59  110  109  110.37  777  108.60  1.74  0.03  103.12  469  101.17   
60000    53  120  119  110.33  772  108.54  1.76  0.03  103.11  468  101.17   
65000    55  130  129  110.30  767  108.48  1.79  0.03  103.08  468  101.09   
70000    51  140  139  110.27  763  108.42  1.81  0.03  103.06  468  101.08   
75000    54  150  149  110.24  758  108.38  1.84  0.03  103.09  469  101.08   
80000    51  160  159  110.21  754  108.33  1.85  0.03  103.07  468  101.07   
85000    54  170  169  110.19  749  108.29  1.87  0.02  103.08  469  101.05   
90000    50  180  179  110.17  746  108.26  1.89  0.02  103.05  468  101.00   
95000    54  190  189  110.15  742  108.22  1.91  0.02  103.22  473  101.16   
100000   55  200  199  110.13  738  108.19  1.92  0.02  103.11  470  101.06   
105000   61  210  209  110.12  734  108.16  1.93  0.02  103.09  469  101.03   
110000   55  220  219  110.10  731  108.14  1.95  0.02  103.08  469  101.01   
115000   58  230  229  110.09  727  108.11  1.96  0.02  103.07  469  100.96   
120000   58  240  239  110.07  724  108.09  1.97  0.02  103.03  468  100.95   
125000   57  250  249  110.06  721  108.06  1.98  0.02  103.02  467  100.92   
130000   56  260  259  110.05  718  108.04  1.99  0.02  102.99  466  100.88   
135000   58  270  269  110.03  716  108.02  2.00  0.02  103.01  467  100.89   
140000   55  280  279  110.02  713  108.00  2.01  0.02  102.98  466  100.86   
145000   55  290  289  110.01  710  107.98  2.02  0.02  103.03  467  100.90   
150000   60  300  299  110.00  708  107.96  2.03  0.02  103.03  467  100.89   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   52  701  200  109.80  656  107.61  2.19  0.01  103.02  467  100.81   
355000   55  711  210  109.80  656  107.60  2.19  0.01  103.00  467  100.77   
360000   52  721  220  109.80  655  107.60  2.19  0.01  103.00  466  100.79   
365000   56  731  230  109.79  654  107.60  2.20  0.01  102.97  466  100.75   
370000   51  741  240  109.79  654  107.59  2.20  0.01  102.98  466  100.76   
375000   66  751  250  109.79  653  107.59  2.20  0.01  102.97  466  100.74   
380000   54  761  260  109.79  653  107.58  2.20  0.01  102.97  465  100.74   
385000   56  771  270  109.78  652  107.58  2.20  0.01  102.95  465  100.71   
390000   62  781  280  109.78  652  107.58  2.21  0.01  102.95  465  100.73   
395000   70  791  290  109.78  651  107.57  2.21  0.01  102.96  465  100.72   
400000   54  801  300  109.78  651  107.57  2.21  0.01  102.96  465  100.75   
405000   56  811  310  109.77  650  107.56  2.21  0.01  102.96  465  100.74   
410000   52  821  320  109.77  650  107.56  2.21  0.01  102.97  465  100.75   
415000   56  831  330  109.77  649  107.56  2.22  0.01  102.97  466  100.72   
420000   52  841  340  109.77  649  107.55  2.22  0.01  102.99  466  100.77   
425000   56  851  350  109.76  648  107.55  2.22  0.01  102.98  466  100.73   
430000   52  861  360  109.76  648  107.55  2.22  0.01  103.00  467  100.79   
435000   71  871  370  109.76  647  107.54  2.22  0.01  102.97  466  100.

0 R: 1.000 P: 0.395 quality price 'm bought ... - time nice 've made
   1 R: 0.295 P: 0.133 pocket room ipad small mouse power charger carry cord netbook
     11 R: 0.087 P: 0.087 pockets carry strap shoulder compartment straps pack back handle space
     13 R: 0.075 P: 0.075 ! love carry perfect recommend school room books highly bought
   4 R: 0.108 P: 0.073 sleeve protection air inside inch protect pro neoprene snug chromebook
     42 R: 0.035 P: 0.035 ; & perfectly sleeve inside pro size hp big dell
   2 R: 0.092 P: 0.051 mac cover pro love perfectly recommend color ! protect hard
     22 R: 0.042 P: 0.042 ! love perfect color perfectly price buy ... cute recommend
   3 R: 0.110 P: 0.071 cover bottom keyboard top screen hard plastic speck easily scratches
     31 R: 0.039 P: 0.039 color picture cover love blue ordered ! received keyboard pink
